In [1]:
import RENT 
import BIC_hyperparameter_search
import numpy as np
import pandas as pd

from sklearn.metrics import f1_score, matthews_corrcoef, accuracy_score, matthews_corrcoef
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

import warnings
warnings.filterwarnings("ignore")

In [2]:
train_data = pd.read_csv('data\\c3\\wisconsin_train.csv').iloc[:,1:]
train_labels = 1 - pd.read_csv('data\\c3\\wisconsin_train_labels.csv').iloc[:,1]
test_data = pd.read_csv('data\\c3\\wisconsin_test.csv').iloc[:,1:]
test_labels = 1 - pd.read_csv('data\\c3\\wisconsin_test_labels.csv').iloc[:,1]

In [3]:
# Define a range of regualarization parameters for the penalty term. A minimum of at least one value is required.
my_C_params = [0.1, 1, 10]

# Define a range of l1-ratios for elastic net.  A minimum of at least one value is required.
my_l1_ratios = [0, 0.1, 0.25, 0.5, 0.75, 0.9, 1]

# Define setting for RENT
model = RENT.RENT_Classification(data=train_data, 
                                 target=train_labels, 
                                 feat_names=train_data.columns, 
                                 C=my_C_params,
                                 l1_ratios=my_l1_ratios,
                                 autoEnetParSel=True,
                                 BIC=True,
                                 poly='OFF',
                                 testsize_range=(0.25,0.25),
                                 scoring='mcc',
                                 classifier='logreg',
                                 K=100,
                                 random_state=0,
                                 verbose=1)

data dimension: (399, 30)  data type: <class 'pandas.core.frame.DataFrame'>
target dimension: (399,)
regularization parameters C: [0.1, 1, 10]
elastic net l1_ratios: [0, 0.1, 0.25, 0.5, 0.75, 0.9, 1]
poly: OFF
number of models in ensemble: 100
random state: 0
verbose: 1


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.


classifier: logreg
scoring: mcc


[Parallel(n_jobs=-1)]: Done   7 out of   7 | elapsed:    0.2s finished


In [4]:
model._BIC_df

,0.1,1.0,10.0
0.00,260.603784,237.937293,235.038652
0.10,251.850404,238.18584,235.054014
0.25,221.010979,226.638275,235.077773
0.50,192.422296,215.515464,235.119253
0.75,193.187215,216.504365,235.162998
0.90,173.269467,211.131411,235.190304
1.00,169.750094,205.529354,235.208948


In [5]:
model.train()

In [6]:
model.get_enet_params()

(0.1, 1.0)

In [7]:
cutoff_parameters = {'t1': np.arange(0.2,1.05,0.05),
                     't2': np.arange(0.2,1.05,0.05),
                     't3': [0.9, 0.95, 0.975, 0.99]}

result = BIC_hyperparameter_search.BIC_hyperparameter_search(model, cutoff_parameters, test_data, test_labels)

In [8]:
BIC = result['BIC']
#indices = np.where(BIC == np.min(BIC))
indices_matrix = np.stack(np.where(BIC == np.min(BIC)), axis=0)
tau1 = np.max(indices_matrix[0,:])
indices_matrix = indices_matrix[:,np.where(indices_matrix[0,:] == tau1)[0]]
tau2 = np.max(indices_matrix[1,:])
indices_matrix = indices_matrix[:,np.where(indices_matrix[1,:] == tau2)[0]]
tau3 = np.max(indices_matrix[2,:])

tau1 = np.round(cutoff_parameters['t1'][tau1], 2)
tau2 = np.round(cutoff_parameters['t2'][tau2], 2)
tau3 = np.round(cutoff_parameters['t3'][tau3], 2)


selected_features = model.select_features(tau_1_cutoff=tau1, tau_2_cutoff=tau2, tau_3_cutoff=tau3)

In [9]:
selected_features

array([ 7, 20, 21, 27], dtype=int64)

In [10]:
sc = StandardScaler()
train_data_sc = sc.fit_transform(train_data.iloc[:, selected_features])
test_data_sc = sc.transform(test_data.iloc[:, selected_features])

# Train model with 
model = LogisticRegression(penalty='none', max_iter=8000, solver="saga", random_state=0).\
        fit(train_data_sc, train_labels)

In [11]:
def predictions():
    print("f1 1: ", f1_score(test_labels, model.predict(test_data_sc)))
    print("f1 0: ", f1_score(1 - test_labels, 1 - model.predict(test_data_sc)))
    print("Accuracy: ", accuracy_score(test_labels, model.predict(test_data_sc)))
    print("Matthews correlation coefficient: ", matthews_corrcoef(test_labels, model.predict(test_data_sc)))

In [12]:
predictions()

f1 1:  0.9865470852017937
f1 0:  0.9743589743589743
Accuracy:  0.9823529411764705
Matthews correlation coefficient:  0.9609891503308582


# Competing methods

In [13]:
from sklearn.svm import SVC
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier
from ReliefF import ReliefF

In [14]:
# M0

sc=StandardScaler()
scaled_data = sc.fit_transform(train_data)

sgd = LogisticRegression(penalty="elasticnet", C=0.1, l1_ratio=1, solver="saga", random_state=0)
sgd.fit(scaled_data, train_labels)
params = np.where(sgd.coef_ != 0)[1]

sc = StandardScaler()
train_data_sc = sc.fit_transform(train_data.iloc[:,params])
test_data_sc = sc.transform(test_data.iloc[:, params])
model = LogisticRegression(penalty='none', max_iter=8000, solver="saga", random_state=0).\
        fit(train_data_sc,train_labels)

predictions()

f1 1:  0.9819819819819819
f1 0:  0.9661016949152542
Accuracy:  0.9764705882352941
Matthews correlation coefficient:  0.9480836768972363


In [15]:
num_features = len(selected_features)

In [16]:
# RFE

svc = SVC(kernel="linear", C=1)
rfe = RFE(estimator=svc, n_features_to_select=num_features, step=1)
rfe.fit(scaled_data, train_labels)
params = np.where(rfe.ranking_ == 1)[0]

sc = StandardScaler()
train_data_sc = sc.fit_transform(train_data.iloc[:,params])
test_data_sc = sc.transform(test_data.iloc[:, params])
model = LogisticRegression(penalty='none', max_iter=8000, solver="saga").\
        fit(train_data_sc,train_labels)

predictions()

f1 1:  0.9515418502202643
f1 0:  0.9026548672566371
Accuracy:  0.9352941176470588
Matthews correlation coefficient:  0.8562174533946231


In [17]:
# reliefF
rfF = ReliefF(n_neighbors=np.round(np.shape(train_data)[0]/3).astype(int), n_features_to_keep=num_features)
rfF.fit(scaled_data, train_labels)
params = rfF.top_features[0:num_features]


sc = StandardScaler()
train_data_sc = sc.fit_transform(train_data.iloc[:,params])
test_data_sc = sc.transform(test_data.iloc[:, params])
model = LogisticRegression(penalty='none', max_iter=8000, solver="saga", random_state=0).\
        fit(train_data_sc,train_labels)

predictions()

f1 1:  0.926605504587156
f1 0:  0.8688524590163935
Accuracy:  0.9058823529411765
Matthews correlation coefficient:  0.7966218885241891


In [18]:
# random forest
clf = RandomForestClassifier(random_state=0)
model = clf.fit(scaled_data, train_labels)
ind = np.where(np.argsort(np.argsort(-1*(model.feature_importances_)))<num_features)[0]

sc = StandardScaler()
train_data_sc = sc.fit_transform(train_data.iloc[:,ind])
test_data_sc = sc.transform(test_data.iloc[:, ind])
model = LogisticRegression(penalty='none', max_iter=8000, solver="saga").\
        fit(train_data_sc,train_labels)

predictions()

f1 1:  0.9589041095890412
f1 0:  0.9256198347107438
Accuracy:  0.9470588235294117
Matthews correlation coefficient:  0.8852187340243354
